# Grouping

Chapter 9 in Python for Data Analysis demonstrates a variety of methods to analyze data via data aggregation and grouping operations. Those are the focus of this session.  Our overall goal for this session is to do Exploratory Data Analysis, which is essentially looking at and probing our data to learn about the patterns we can discover in them.  Often this can generate a better understanding of problems in the data, as well as revealing relationships that might be worth exploring in more depth.


For this exercise, we will use 2010 US Census data, for all census tracts in Kentucky.  A good starting point for any kind of Census data is the American Fact Finder: 

https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml

The Census Bureau produces several products, the most famous of which is the decennial census, which as its name implies, is a true Census.  There are a limited number of data fields for the whole population (or at least those that the Census workers are able to reach, which is pretty good).  The Census has other products that are surveys, such as the American Community Survey (ACS) which asks more detailed questions on an annual basis from a 1% sample of households.  

In this case, we are working with file DP-1: Profile of General Population and Housing Characteristics: 2010, which is also a part of something called Summary File 1 (SF-1).  It is for the 100% sample.  A data dictionary for selected fields is below.  

In [1]:
import pandas as pd

# skip the second row, which contains descriptions
sf1 = pd.read_csv('data/DEC_10_SF1_combined.csv', skiprows=[1])

# make the tract, county and state separate
sf1['tract'] = sf1['GEO.display-label'].apply(lambda x: x.split(sep=',')[0])
sf1['county'] = sf1['GEO.display-label'].apply(lambda x: x.split(sep=',')[1])
sf1['state'] = sf1['GEO.display-label'].apply(lambda x: x.split(sep=',')[2])

# compute some derived fields
sf1['pct_rent'] = sf1['H4-D004'] / sf1['H4-D001'] * 100
sf1['pct_black'] = sf1['P3-D003'] / sf1['P3-D001'] * 100
sf1['pct_asian'] = sf1['P3-D005'] / sf1['P3-D001'] * 100
sf1['pct_white'] = sf1['P3-D002'] / sf1['P3-D001'] * 100
sf1['pct_hisp'] = sf1['P4-D003'] / sf1['P4-D001'] * 100
sf1['pct_vacant'] = sf1['H5-D001'] / sf1['H1-D001'] * 100
sf1[:5]

,GEO.id,GEO.id2,GEO.display-label,P1-D001,P3-D001,P3-D002,P3-D003,P3-D004,P3-D005,P3-D006,...,H5-D008,tract,county,state,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant
0,1400000US21001970100,21001970100,"Census Tract 9701, Adair County, Kentucky",1727,1727,1683,14,1,0,1,...,60,Census Tract 9701,Adair County,Kentucky,17.411402,0.810654,0.000000,97.452229,2.142444,16.794872
1,1400000US21001970200,21001970200,"Census Tract 9702, Adair County, Kentucky",1722,1722,1635,41,5,0,0,...,35,Census Tract 9702,Adair County,Kentucky,19.252874,2.380952,0.000000,94.947735,2.613240,25.161290
2,1400000US21001970300,21001970300,"Census Tract 9703, Adair County, Kentucky",3016,3016,2944,6,11,8,0,...,106,Census Tract 9703,Adair County,Kentucky,20.521173,0.198939,0.265252,97.612732,1.856764,15.193370
3,1400000US21001970401,21001970401,"Census Tract 9704.01, Adair County, Kentucky",4070,4070,3716,237,1,16,1,...,109,Census Tract 9704.01,Adair County,Kentucky,37.215909,5.823096,0.393120,91.302211,1.547912,10.795743
4,1400000US21001970402,21001970402,"Census Tract 9704.02, Adair County, Kentucky",4261,4261,3950,180,16,16,3,...,70,Census Tract 9704.02,Adair County,Kentucky,30.911681,4.224360,0.375499,92.701244,1.900962,11.642542


## Groupby and Aggregation Operations

Groupby is a powerful method in pandas that follows the split-apply-combine approach to data.  As shown in Figure 9-1 in the context of a sum operation, the data is first split into groups that share the same key values.  Then an operation, in this case a sum, is applied to each group.  Then the results are combined.

The built-in aggregation methods available for groupby operations include:
* count
* sum
* mean
* median
* std, var
* min, max
* first, last

You can also apply your own functions as aggregation methods.

![Groupby Operations](groupby.png "Groupby")

Let's apply this approach to computing total population in each county in our dataset.  We can do this in two steps to help explain what is happening.  First we create a groupby object, using county codes to group all the census blocks in sf1 into groups that share the same county code.

In [7]:
grouped = sf1['P1-D001'].groupby(sf1['county'])
grouped.head()

0       1727
1       1722
2       3016
3       4070
4       4261
        ... 
1107    3144
1108    3927
1109    3139
1110    3261
1111    3757
Name: P1-D001, Length: 504, dtype: int64

Now that we have this grouping object that represents the **split** part of the workflow in the figure above, we can **apply** operations and **combine** the results using methods like sum:

In [13]:
grouped.sum()

county
 Adair County       18656
 Allen County       19956
 Anderson County    21421
 Ballard County      8249
 Barren County      42173
                    ...  
 Wayne County       20813
 Webster County     13621
 Whitley County     35637
 Wolfe County        7355
 Woodford County    24939
Name: P1-D001, Length: 120, dtype: int64

We might want to capture the result in a DataFrame if we want to use it in other processing, like merging the results to the original DataFrame.

In [8]:
county_pop = sf1['P1-D001'].groupby(sf1['county']).sum().to_frame(name='county_population')
county_pop

,county_population
county,
Adair County,18656
Allen County,19956
Anderson County,21421
Ballard County,8249
Barren County,42173
...,...
Wayne County,20813
Webster County,13621
Whitley County,35637


Here we merge the county total population with sf1 and create a new DataFrame.

In [10]:
sf2 = pd.merge(sf1,county_pop, left_on='county', right_index=True)
county_pop = sf1['P1-D001'].groupby(sf1['county']).sum().to_frame(name='county_population')
sf2[:10]

,GEO.id,GEO.id2,GEO.display-label,P1-D001,P3-D001,P3-D002,P3-D003,P3-D004,P3-D005,P3-D006,...,tract,county,state,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant,county_population
0,1400000US21001970100,21001970100,"Census Tract 9701, Adair County, Kentucky",1727,1727,1683,14,1,0,1,...,Census Tract 9701,Adair County,Kentucky,17.411402,0.810654,0.000000,97.452229,2.142444,16.794872,18656
1,1400000US21001970200,21001970200,"Census Tract 9702, Adair County, Kentucky",1722,1722,1635,41,5,0,0,...,Census Tract 9702,Adair County,Kentucky,19.252874,2.380952,0.000000,94.947735,2.613240,25.161290,18656
2,1400000US21001970300,21001970300,"Census Tract 9703, Adair County, Kentucky",3016,3016,2944,6,11,8,0,...,Census Tract 9703,Adair County,Kentucky,20.521173,0.198939,0.265252,97.612732,1.856764,15.193370,18656
3,1400000US21001970401,21001970401,"Census Tract 9704.01, Adair County, Kentucky",4070,4070,3716,237,1,16,1,...,Census Tract 9704.01,Adair County,Kentucky,37.215909,5.823096,0.393120,91.302211,1.547912,10.795743,18656
4,1400000US21001970402,21001970402,"Census Tract 9704.02, Adair County, Kentucky",4261,4261,3950,180,16,16,3,...,Census Tract 9704.02,Adair County,Kentucky,30.911681,4.224360,0.375499,92.701244,1.900962,11.642542,18656
5,1400000US21001970500,21001970500,"Census Tract 9705, Adair County, Kentucky",2457,2457,2402,16,2,5,0,...,Census Tract 9705,Adair County,Kentucky,17.972832,0.651201,0.203500,97.761498,0.773301,17.641997,18656
6,1400000US21001970600,21001970600,"Census Tract 9706, Adair County, Kentucky",1403,1403,1366,9,0,0,0,...,Census Tract 9706,Adair County,Kentucky,16.920474,0.641483,0.000000,97.362794,1.282965,13.848397,18656
7,1400000US21003920100,21003920100,"Census Tract 9201, Allen County, Kentucky",3185,3185,3114,16,8,4,0,...,Census Tract 9201,Allen County,Kentucky,15.139752,0.502355,0.125589,97.770801,1.098901,27.559055,19956
8,1400000US21003920200,21003920200,"Census Tract 9202, Allen County, Kentucky",2703,2703,2645,6,11,8,0,...,Census Tract 9202,Allen County,Kentucky,15.276273,0.221976,0.295967,97.854236,1.479837,12.178877,19956
9,1400000US21003920300,21003920300,"Census Tract 9203, Allen County, Kentucky",4685,4685,4473,83,19,12,0,...,Census Tract 9203,Allen County,Kentucky,34.676186,1.771612,0.256137,95.474920,1.173959,9.254975,19956


Your turn:  Instead of grouping by the county, group by the state.  What is the total population of Kentucky?  Is there a simple way to check this?  

In [11]:
grouped_state = sf1['P1-D001'].groupby(sf1['state'])

In [12]:
grouped_state.sum()

state
 Kentucky    4339367
Name: P1-D001, dtype: int64

## Transforming Data with Groupby

In some cases you may want to apply a function to your data, by group.  An example would be to normalize a column by a mean of each group.  Say we wanted to subtract the mean population density of each county from the population density of each census block. We could write a function to subtract the mean from each value, and then use the transform operation to apply this to each group:

In [15]:
def demean(arr):
    return arr - arr.mean()

Now we can apply this tranformation to columns in our dataframe.  As examples, let's 'demean' the pct_black and pct_rent columns, subtracting the county-wide mean of these values from the tract-specific values, so that the result is transformed to have a mean of zero within each county.

To check the results, we print the means per county, then the original values for the first 5 rows, then the transformed results.  The transformed results we should be able to calculate by subtracting the appropriate county mean from the tract value.

In [16]:
normalized = sf1[['pct_black', 'pct_rent']].groupby(sf1['county']).transform(demean)
print(sf1[['pct_black', 'pct_rent']].groupby(sf1['county']).mean())
print(sf1[['county','pct_black', 'pct_rent']][:5])
print(normalized[:5])
# sf1['pct_rent'] = sf1['H4-D004'] / sf1['H4-D001'] * 100
# sf1['pct_black'] = sf1['P3-D003'] / sf1['P3-D001'] * 100
# sf1['pct_asian'] = sf1['P3-D005'] / sf1['P3-D001'] * 100
# sf1['pct_white'] = sf1['P3-D002'] / sf1['P3-D001'] * 100
# sf1['pct_hisp'] = sf1['P4-D003'] / sf1['P4-D001'] * 100
# sf1['pct_vacant'] = sf1['H5-D001'] / sf1['H1-D001'] * 100


                  pct_black   pct_rent
county                                
 Adair County      2.104384  22.886621
 Allen County      0.716792  22.312658
 Anderson County   1.419188  20.908477
 Ballard County    2.690326  21.350937
 Barren County     3.385357  29.982273
...                     ...        ...
 Wayne County      1.288175  25.683905
 Webster County    3.825768  22.732870
 Whitley County    0.519283  31.299399
 Wolfe County      0.131572  27.972188
 Woodford County   4.802279  29.615644

[120 rows x 2 columns]
          county  pct_black   pct_rent
0   Adair County   0.810654  17.411402
1   Adair County   2.380952  19.252874
2   Adair County   0.198939  20.521173
3   Adair County   5.823096  37.215909
4   Adair County   4.224360  30.911681
   pct_black   pct_rent
0  -1.293729  -5.475218
1   0.276569  -3.633747
2  -1.905445  -2.365448
3   3.718712  14.329289
4   2.119977   8.025060


We can merge these transformed results on to the original DataFrame, and check the means of the original variables and the tranformed ones.  The transformed ones should be arbitrarily close to zero.

In [29]:
sf2 = pd.merge(sf1,normalized, left_index=True, right_index=True)
sf2[:5]
# sf2.groupby('county')[['pct_black_x', 'pct_black_y', 'pct_rent_x', 'pct_rent_y']].mean()

,GEO.id,GEO.id2,GEO.display-label,P1-D001,P3-D001,P3-D002,P3-D003,P3-D004,P3-D005,P3-D006,...,county,state,pct_rent_x,pct_black_x,pct_asian,pct_white,pct_hisp,pct_vacant,pct_black_y,pct_rent_y
0,1400000US21001970100,21001970100,"Census Tract 9701, Adair County, Kentucky",1727,1727,1683,14,1,0,1,...,Adair County,Kentucky,17.411402,0.810654,0.000000,97.452229,2.142444,16.794872,-1.293729,-5.475218
1,1400000US21001970200,21001970200,"Census Tract 9702, Adair County, Kentucky",1722,1722,1635,41,5,0,0,...,Adair County,Kentucky,19.252874,2.380952,0.000000,94.947735,2.613240,25.161290,0.276569,-3.633747
2,1400000US21001970300,21001970300,"Census Tract 9703, Adair County, Kentucky",3016,3016,2944,6,11,8,0,...,Adair County,Kentucky,20.521173,0.198939,0.265252,97.612732,1.856764,15.193370,-1.905445,-2.365448
3,1400000US21001970401,21001970401,"Census Tract 9704.01, Adair County, Kentucky",4070,4070,3716,237,1,16,1,...,Adair County,Kentucky,37.215909,5.823096,0.393120,91.302211,1.547912,10.795743,3.718712,14.329289
4,1400000US21001970402,21001970402,"Census Tract 9704.02, Adair County, Kentucky",4261,4261,3950,180,16,16,3,...,Adair County,Kentucky,30.911681,4.224360,0.375499,92.701244,1.900962,11.642542,2.119977,8.025060


Apply is a method we have learned previously, which allows us to apply a function to each row in a DataFrame.  We can also combine apply with groupby to apply functions based on group membership.  For example, the function 'top' sorts an array and selects the top n rows from it.  We provide some defaults for the arguments of how many rows, and the column to use for the selection:

In [ ]:
def top(df, n=5, column='pct_rent'):
    return df.sort_values(by=column, ascending=False).head(n)

Using this on the full dataset and setting the number of rows and the column to get the top values for, in this case using pct_rent to override the default argument, we get the top 10 tracts in the region in terms of percentage rental.

In [ ]:
top(sf1, n=10, column='pct_rent')

Below we apply this with groupby and use the defaults for n and column, and it applies the function within each county and concatenates the results, producing the top 5 blocks on pop_sqmi for each county in the region.

In [ ]:
sf1.groupby('county').apply(top)

Here we pass arguments to the function to set n and the column to select the top value from.

In [ ]:
sf1.groupby('county').apply(top, n=1, column='P1-D001')

## Exploring Correlations in the Data

Pandas provides simple ways of computing correlation coefficients among the columns in your DataFrame.  If you use corr() on a full DF, it will produce a large correlation table.  A bit hard to navigate and you mostly would not be interested in some of these results.

In [ ]:
sf1.corr()

It is easy to compute correlation coeffients for a subset of columns.

In [18]:
sf1[['pct_rent', 'pct_vacant']].corr()

,pct_rent,pct_vacant
pct_rent,1.000000,0.085586
pct_vacant,0.085586,1.000000


And this method can be combined with groupby to compute correlation tables by group.

In [19]:
sf1.groupby('county')[['pct_rent', 'pct_vacant']].corr()

pct_rent  pct_vacant
county                                           
 Adair County    pct_rent    1.000000   -0.613912
                 pct_vacant -0.613912    1.000000
 Allen County    pct_rent    1.000000   -0.620421
                 pct_vacant -0.620421    1.000000
 Anderson County pct_rent    1.000000   -0.534183
...                               ...         ...
 Whitley County  pct_vacant  0.483214    1.000000
 Wolfe County    pct_rent    1.000000    1.000000
                 pct_vacant  1.000000    1.000000
 Woodford County pct_rent    1.000000    0.154547
                 pct_vacant  0.154547    1.000000

[240 rows x 2 columns]

## Your turn to practice:

Count the number of census blocks per county.

Calculate total households per county.

Calculate percent renters by county. (Careful not to calculate the mean percent rental across blocks in a county)

Calculate percent vacant by county.

Calculate mean, min and max vacancy rate (at the block level) by county.

Calculate the 90th percentile of vacancy rate (at the block level) by county.

In [23]:
num_of_blocks = sf1['P1-D001'].groupby(sf1['county']).count()
num_of_blocks

county
 Adair County        7
 Allen County        6
 Anderson County     5
 Ballard County      3
 Barren County      10
                    ..
 Wayne County        5
 Webster County      4
 Whitley County      8
 Wolfe County        2
 Woodford County     8
Name: P1-D001, Length: 120, dtype: int64

In [24]:
sum_of_blocks = sf1['P1-D001'].groupby(sf1['county']).sum()
sum_of_blocks

county
 Adair County       18656
 Allen County       19956
 Anderson County    21421
 Ballard County      8249
 Barren County      42173
                    ...  
 Wayne County       20813
 Webster County     13621
 Whitley County     35637
 Wolfe County        7355
 Woodford County    24939
Name: P1-D001, Length: 120, dtype: int64

In [66]:
temp_result = sf1['H1-D001'].groupby(sf1['county']).sum().to_frame('county_pop')
sf3 = pd.merge(sf1,temp_result, left_on='county', right_index=True)
sf3[['county','county_pop','H4-D004']][:10]

sf3['rent_pct'] = sf3['H4-D004'] / sf3['county_pop'] * 100
sf3[['county','county_pop','H4-D004', 'rent_pct']][:10]

,county,county_pop,H4-D004,rent_pct
0,Adair County,7285,113,1.551132
1,Adair County,7285,134,1.839396
2,Adair County,7285,252,3.459163
3,Adair County,7285,655,8.991078
4,Adair County,7285,434,5.957447
5,Adair County,7285,172,2.361016
6,Adair County,7285,100,1.372684
7,Allen County,7848,195,2.484709
8,Allen County,7848,141,1.796636
9,Allen County,7848,680,8.664628


In [98]:
temp_vacant = sf1['H1-D001'].groupby(sf1['county']).sum().to_frame('county_vacant')
temp_vacant_per_county = sf1['H3-D003'].groupby(sf1['county']).sum().to_frame('vacant_per_county')

sf4 = pd.merge(sf1,temp_vacant, left_on='county', right_index=True)
sf4[:5]
sf4 = pd.merge(sf4,temp_vacant_per_county, left_on='county', right_index=True)
sf4[:4]
sf4[['county','county_vacant','vacant_per_county']][:10]
sf4['vacant_pct'] = sf4['vacant_per_county'] / sf4['county_vacant'] * 100
sf4[['county','county_vacant','vacant_per_county', 'vacant_pct']][:10]

,county,county_vacant,vacant_per_county,vacant_pct
0,Adair County,8568,1283,14.974323
1,Adair County,8568,1283,14.974323
2,Adair County,8568,1283,14.974323
3,Adair County,8568,1283,14.974323
4,Adair County,8568,1283,14.974323
5,Adair County,8568,1283,14.974323
6,Adair County,8568,1283,14.974323
7,Allen County,9307,1459,15.676373
8,Allen County,9307,1459,15.676373
9,Allen County,9307,1459,15.676373


In [112]:
sf4['vacant_pct_block'] = sf4['H5-D001'] / sf4['vacant_per_county'] * 100
print(sf4[['county','H5-D001','vacant_per_county', 'vacant_pct_block']][:10])
mean_ = sf4['vacant_pct_block'].groupby(sf4['county']).mean().to_frame('mean')
print(mean_)
min_ = sf4['vacant_pct_block'].groupby(sf4['county']).min().to_frame('min')
print(min_)
max_ = sf4['vacant_pct_block'].groupby(sf4['county']).max().to_frame('max')
print(max_)

          county  H5-D001  vacant_per_county  vacant_pct_block
0   Adair County      131               1283         10.210444
1   Adair County      234               1283         18.238504
2   Adair County      220               1283         17.147311
3   Adair County      213               1283         16.601715
4   Adair County      185               1283         14.419330
5   Adair County      205               1283         15.978176
6   Adair County       95               1283          7.404521
7   Allen County      490               1459         33.584647
8   Allen County      128               1459          8.773132
9   Allen County      200               1459         13.708019
                       mean
county                     
 Adair County     14.285714
 Allen County     16.666667
 Anderson County  20.000000
 Ballard County   33.333333
 Barren County    10.000000
...                     ...
 Wayne County     20.000000
 Webster County   25.000000
 Whitley County   12.500000

In [118]:
print(sf4['vacant_pct_block'].groupby(sf4['county']).quantile(0.9))

county
 Adair County       17.583788
 Allen County       27.107608
 Anderson County    31.345646
 Ballard County     45.122951
 Barren County      15.267245
                      ...    
 Wayne County       40.932056
 Webster County     35.828313
 Whitley County     15.625393
 Wolfe County       55.714286
 Woodford County    15.712707
Name: vacant_pct_block, Length: 120, dtype: float64


                 GEO.id      GEO.id2  \
0  1400000US21001970100  21001970100   
1  1400000US21001970200  21001970200   
2  1400000US21001970300  21001970300   
3  1400000US21001970401  21001970401   
4  1400000US21001970402  21001970402   
5  1400000US21001970500  21001970500   
6  1400000US21001970600  21001970600   
7  1400000US21003920100  21003920100   
8  1400000US21003920200  21003920200   
9  1400000US21003920300  21003920300   

                              GEO.display-label  P1-D001  P3-D001  P3-D002  \
0     Census Tract 9701, Adair County, Kentucky     1727     1727     1683   
1     Census Tract 9702, Adair County, Kentucky     1722     1722     1635   
2     Census Tract 9703, Adair County, Kentucky     3016     3016     2944   
3  Census Tract 9704.01, Adair County, Kentucky     4070     4070     3716   
4  Census Tract 9704.02, Adair County, Kentucky     4261     4261     3950   
5     Census Tract 9705, Adair County, Kentucky     2457     2457     2402   
6     Census 